# Transliteration Model Testing

This notebook tests the Romanized Kannada to Kannada script transliteration functionality.

## Contents
1. Setup and Imports
2. Basic Transliteration Tests
3. Side-by-Side Comparison
4. Accuracy Calculation with Reference Translations
5. Error Pattern Analysis
6. Visualization of Results

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
from typing import List, Dict, Tuple

# Import project modules
from src.preprocessing import Transliterator, TransliterationResult
from src.preprocessing import LanguageDetector

print("Imports successful!")

In [ ]:
# Initialize the transliterator
transliterator = Transliterator()

# Check status
status = transliterator.get_status()
print("Transliterator Status:")
for key, value in status.items():
    print(f"  {key}: {value}")

## 2. Basic Transliteration Tests

In [ ]:
# Test data: Romanized Kannada samples
test_samples = [
    # Greetings
    "namaskara",
    "namaskara hegidira",
    
    # Positive sentiments
    "tumba chennagide",
    "ee product tumba olle",
    "nanna ishta aagide",
    "quality tumba chennagide",
    
    # Negative sentiments
    "bekilla ee product",
    "tumba ketta quality",
    "sariyilla beda",
    
    # Mixed/Neutral
    "ok agide",
    "price tumba bele",
    "delivery fast aagide",
    
    # Questions
    "yenu price ide",
    "elli sigatte",
    "yaake delay aagide",
    
    # Longer sentences
    "ee product tumba chennagide naanu recommend madtini",
    "quality olle aadare price tumba bele",
]

print(f"Total test samples: {len(test_samples)}")

In [ ]:
# Perform transliteration on all samples
results = []

for sample in test_samples:
    result = transliterator.transliterate(sample)
    results.append({
        'input': sample,
        'output': result.transliterated,
        'method': result.method,
        'word_count': len(sample.split()),
        'mappings': result.word_mappings
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
print("Transliteration completed!")
print(f"Results shape: {results_df.shape}")

## 3. Side-by-Side Comparison

In [ ]:
def display_comparison(df: pd.DataFrame, title: str = "Transliteration Results"):
    """
    Display a formatted side-by-side comparison of inputs and outputs.
    """
    print("=" * 100)
    print(f" {title}")
    print("=" * 100)
    print()
    
    for idx, row in df.iterrows():
        print(f"[{idx + 1}] Input:  {row['input']}")
        print(f"    Output: {row['output']}")
        print(f"    Method: {row['method']} | Words: {row['word_count']}")
        print("-" * 80)

# Display all results
display_comparison(results_df)

In [ ]:
# Create a styled DataFrame for better visualization
display_df = results_df[['input', 'output', 'method']].copy()
display_df.columns = ['Romanized Input', 'Kannada Output', 'Method']
display_df.index = range(1, len(display_df) + 1)
display_df.index.name = '#'

# Display with styling
display_df.style.set_properties(**{
    'text-align': 'left',
    'font-size': '14px'
}).set_table_styles([
    {'selector': 'th', 'props': [('font-size', '14px'), ('text-align', 'center')]}
])

## 4. Accuracy Calculation with Reference Translations

We'll calculate accuracy by comparing transliteration output with known correct translations.

In [ ]:
# Reference translations (ground truth)
reference_data = [
    # (romanized, expected_kannada)
    ("namaskara", "ನಮಸ್ಕಾರ"),
    ("hegidira", "ಹೇಗಿದೀರ"),
    ("tumba", "ತುಂಬ"),
    ("chennagide", "ಚೆನ್ನಾಗಿದೆ"),
    ("olle", "ಒಳ್ಳೆ"),
    ("olleya", "ಒಳ್ಳೆಯ"),
    ("ketta", "ಕೆಟ್ಟ"),
    ("bekilla", "ಬೇಕಿಲ್ಲ"),
    ("beda", "ಬೇಡ"),
    ("illa", "ಇಲ್ಲ"),
    ("ide", "ಇದೆ"),
    ("agide", "ಆಗಿದೆ"),
    ("beku", "ಬೇಕು"),
    ("gottu", "ಗೊತ್ತು"),
    ("naanu", "ನಾನು"),
    ("neenu", "ನೀನು"),
    ("avaru", "ಅವರು"),
    ("idu", "ಇದು"),
    ("adu", "ಅದು"),
    ("yenu", "ಯೇನು"),
    ("elli", "ಎಲ್ಲಿ"),
    ("hege", "ಹೇಗೆ"),
    ("yaake", "ಯಾಕೆ"),
    ("mattu", "ಮತ್ತು"),
    ("aadare", "ಆದರೆ"),
    ("santhosha", "ಸಂತೋಷ"),
    ("dhanyavada", "ಧನ್ಯವಾದ"),
    ("bengaluru", "ಬೆಂಗಳೂರು"),
    ("kannada", "ಕನ್ನಡ"),
    ("amma", "ಅಮ್ಮ"),
]

print(f"Reference data: {len(reference_data)} word pairs")

In [ ]:
def calculate_accuracy(transliterator: Transliterator, 
                       reference_data: List[Tuple[str, str]]) -> Dict:
    """
    Calculate transliteration accuracy against reference translations.
    
    Returns:
        Dictionary with accuracy metrics and detailed results.
    """
    correct = 0
    incorrect = []
    results = []
    
    for romanized, expected in reference_data:
        output, method = transliterator.transliterate_word(romanized)
        is_correct = output == expected
        
        if is_correct:
            correct += 1
        else:
            incorrect.append({
                'input': romanized,
                'expected': expected,
                'got': output,
                'method': method
            })
        
        results.append({
            'input': romanized,
            'expected': expected,
            'output': output,
            'correct': is_correct,
            'method': method
        })
    
    total = len(reference_data)
    accuracy = correct / total if total > 0 else 0
    
    return {
        'total': total,
        'correct': correct,
        'incorrect': len(incorrect),
        'accuracy': accuracy,
        'accuracy_percent': f"{accuracy * 100:.2f}%",
        'errors': incorrect,
        'detailed_results': results
    }

# Calculate accuracy
accuracy_results = calculate_accuracy(transliterator, reference_data)

print("=" * 50)
print(" ACCURACY REPORT")
print("=" * 50)
print(f"Total words tested: {accuracy_results['total']}")
print(f"Correct: {accuracy_results['correct']}")
print(f"Incorrect: {accuracy_results['incorrect']}")
print(f"Accuracy: {accuracy_results['accuracy_percent']}")
print("=" * 50)

In [ ]:
# Display detailed results as a DataFrame
accuracy_df = pd.DataFrame(accuracy_results['detailed_results'])
accuracy_df['status'] = accuracy_df['correct'].map({True: '✓', False: '✗'})

# Show the results with styling
def highlight_errors(row):
    if row['correct']:
        return ['background-color: #d4edda'] * len(row)
    else:
        return ['background-color: #f8d7da'] * len(row)

styled_df = accuracy_df[['input', 'expected', 'output', 'status', 'method']].style.apply(
    highlight_errors, axis=1
)
styled_df

In [ ]:
# Display errors in detail
if accuracy_results['errors']:
    print("\nERROR DETAILS:")
    print("-" * 70)
    for error in accuracy_results['errors']:
        print(f"Input:    {error['input']}")
        print(f"Expected: {error['expected']}")
        print(f"Got:      {error['got']}")
        print(f"Method:   {error['method']}")
        print("-" * 70)
else:
    print("\n✓ No errors! All transliterations matched expected output.")

## 5. Error Pattern Analysis

Analyze patterns in transliteration errors to identify areas for improvement.

In [ ]:
# Test with words NOT in the fallback dictionary to analyze error patterns
unknown_words = [
    # Words likely NOT in fallback dict
    ("preetiya", "ಪ್ರೀತಿಯ"),
    ("kathe", "ಕಥೆ"),
    ("pustaka", "ಪುಸ್ತಕ"),
    ("shale", "ಶಾಲೆ"),
    ("makkalu", "ಮಕ್ಕಳು"),
    ("hesaru", "ಹೆಸರು"),
    ("kelasa", "ಕೆಲಸ"),
    ("dina", "ದಿನ"),
    ("samaya", "ಸಮಯ"),
    ("jagat", "ಜಗತ್"),
    ("vishaya", "ವಿಷಯ"),
    ("karnataka", "ಕರ್ನಾಟಕ"),
    ("bharata", "ಭಾರತ"),
    ("cinema", "ಸಿನಿಮಾ"),
    ("sangeetha", "ಸಂಗೀತ"),
]

# Test these words
unknown_results = calculate_accuracy(transliterator, unknown_words)

print("Unknown Words Test Results:")
print(f"Accuracy: {unknown_results['accuracy_percent']}")
print(f"Words handled: {unknown_results['correct']}/{unknown_results['total']}")

In [ ]:
def analyze_error_patterns(errors: List[Dict]) -> Dict:
    """
    Analyze error patterns in transliteration failures.
    """
    if not errors:
        return {'message': 'No errors to analyze'}
    
    patterns = {
        'unchanged': [],  # Words returned as-is (not transliterated)
        'partial': [],    # Words partially transliterated
        'wrong': [],      # Words transliterated but incorrectly
    }
    
    # Character patterns that often cause issues
    problem_chars = Counter()
    problem_endings = Counter()
    
    for error in errors:
        input_word = error['input']
        expected = error['expected']
        got = error['got']
        
        # Categorize error type
        if got == input_word:
            patterns['unchanged'].append(error)
        elif any(ord(c) >= 0x0C80 and ord(c) <= 0x0CFF for c in got):
            patterns['wrong'].append(error)
        else:
            patterns['partial'].append(error)
        
        # Analyze problematic character patterns
        for char in input_word:
            problem_chars[char] += 1
        
        # Analyze word endings
        if len(input_word) >= 2:
            problem_endings[input_word[-2:]] += 1
    
    return {
        'patterns': patterns,
        'unchanged_count': len(patterns['unchanged']),
        'partial_count': len(patterns['partial']),
        'wrong_count': len(patterns['wrong']),
        'common_problem_chars': problem_chars.most_common(10),
        'common_problem_endings': problem_endings.most_common(5),
    }

# Analyze errors from unknown words test
error_analysis = analyze_error_patterns(unknown_results['errors'])

print("Error Pattern Analysis:")
print("=" * 50)
print(f"Unchanged (not transliterated): {error_analysis['unchanged_count']}")
print(f"Partial transliteration: {error_analysis['partial_count']}")
print(f"Wrong transliteration: {error_analysis['wrong_count']}")
print()
print("Most common characters in failed words:")
for char, count in error_analysis['common_problem_chars'][:5]:
    print(f"  '{char}': {count} occurrences")
print()
print("Most common endings in failed words:")
for ending, count in error_analysis['common_problem_endings']:
    print(f"  '{ending}': {count} occurrences")

In [ ]:
# Display unchanged words (these need to be added to fallback dict)
if error_analysis['patterns']['unchanged']:
    print("\nWords that need to be added to fallback dictionary:")
    print("-" * 60)
    for error in error_analysis['patterns']['unchanged']:
        print(f'    "{error["input"]}": "{error["expected"]}",')  # Ready to copy-paste!

## 6. Visualization of Results

In [ ]:
# Create accuracy visualization
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'bar'}]],
    subplot_titles=('Transliteration Accuracy', 'Error Types Distribution')
)

# Pie chart for overall accuracy
labels = ['Correct', 'Incorrect']
values = [accuracy_results['correct'], accuracy_results['incorrect']]
colors = ['#28a745', '#dc3545']

fig.add_trace(
    go.Pie(
        labels=labels,
        values=values,
        marker_colors=colors,
        hole=0.4,
        textinfo='label+percent',
        textposition='outside'
    ),
    row=1, col=1
)

# Bar chart for error types (from unknown words test)
error_types = ['Unchanged', 'Partial', 'Wrong']
error_counts = [
    error_analysis['unchanged_count'],
    error_analysis['partial_count'],
    error_analysis['wrong_count']
]

fig.add_trace(
    go.Bar(
        x=error_types,
        y=error_counts,
        marker_color=['#ffc107', '#17a2b8', '#dc3545'],
        text=error_counts,
        textposition='auto'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='Transliteration Performance Analysis',
    showlegend=False,
    height=400
)

fig.show()

In [ ]:
# Visualize method distribution
method_counts = results_df['method'].value_counts()

fig = px.pie(
    values=method_counts.values,
    names=method_counts.index,
    title='Transliteration Method Distribution',
    color_discrete_sequence=px.colors.qualitative.Set2,
    hole=0.3
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=400)
fig.show()

In [ ]:
# Visualize character frequency in test data
all_chars = ''.join(test_samples).lower()
char_freq = Counter(c for c in all_chars if c.isalpha())

# Get top 15 characters
top_chars = char_freq.most_common(15)

fig = px.bar(
    x=[c[0] for c in top_chars],
    y=[c[1] for c in top_chars],
    title='Character Frequency in Test Samples',
    labels={'x': 'Character', 'y': 'Frequency'},
    color=[c[1] for c in top_chars],
    color_continuous_scale='Viridis'
)

fig.update_layout(
    xaxis_title='Character',
    yaxis_title='Frequency',
    height=400,
    showlegend=False
)

fig.show()

In [ ]:
# Word length analysis
results_df['input_length'] = results_df['input'].apply(len)
results_df['output_length'] = results_df['output'].apply(len)

fig = px.scatter(
    results_df,
    x='input_length',
    y='output_length',
    color='method',
    size='word_count',
    hover_data=['input', 'output'],
    title='Input vs Output Length by Method',
    labels={
        'input_length': 'Input Length (chars)',
        'output_length': 'Output Length (chars)',
        'method': 'Method'
    }
)

# Add diagonal line for reference
max_len = max(results_df['input_length'].max(), results_df['output_length'].max())
fig.add_trace(
    go.Scatter(
        x=[0, max_len],
        y=[0, max_len],
        mode='lines',
        line=dict(dash='dash', color='gray'),
        name='1:1 ratio'
    )
)

fig.update_layout(height=500)
fig.show()

## 7. Summary and Recommendations

In [ ]:
# Generate summary report
print("=" * 70)
print(" TRANSLITERATION MODEL TEST SUMMARY")
print("=" * 70)
print()
print("CONFIGURATION:")
print(f"  Model available: {transliterator.model_available}")
print(f"  Fallback dictionary size: {len(transliterator.fallback_dict)} words")
print()
print("ACCURACY METRICS:")
print(f"  Known words accuracy: {accuracy_results['accuracy_percent']}")
print(f"  Unknown words accuracy: {unknown_results['accuracy_percent']}")
print()
print("ERROR ANALYSIS:")
print(f"  Unchanged words: {error_analysis['unchanged_count']}")
print(f"  Wrong translations: {error_analysis['wrong_count']}")
print()
print("RECOMMENDATIONS:")
if error_analysis['unchanged_count'] > 0:
    print("  1. Add missing words to fallback dictionary")
if not transliterator.model_available:
    print("  2. Consider using Python 3.10 to enable IndicXlit model")
print("  3. Collect more training data for domain-specific words")
print("  4. Test with real user reviews for better coverage")
print()
print("=" * 70)

In [ ]:
# Interactive testing cell - modify the text below to test your own inputs
test_text = "namaskara neevu hegidira tumba chennagide"

result = transliterator.transliterate(test_text)
print(f"Input:  {result.original}")
print(f"Output: {result.transliterated}")
print(f"Method: {result.method}")
print()
print("Word mappings:")
for original, translated in result.word_mappings:
    print(f"  {original} → {translated}")